In [1]:
import statisUtils as utils
import pandas as pd
import numpy as np
import random
from scipy.stats import loguniform, lognorm
from scipy import stats
import time
import math

import ABCSampler as abcSampler


In [2]:
# df = pd.read_csv('data/MEF_QC_all.csv', header=None) # gene x cells
df = pd.read_csv('data/outlier_genes_3.csv', header=None) # gene x cells

data_all = np.array(df, dtype=int)
data_all

array([[ 66,  95,  50, ...,  32,  39,  49],
       [ 38,  94,  75, ...,  75,  40,  52],
       [  7,   9,  13, ...,   3,   0,   6],
       ...,
       [  4,   7,   3, ...,   0,   0,   4],
       [  0,   0,   0, ...,  24,   0,   5],
       [ 85,  65, 136, ...,  84,  43,  51]])

In [3]:
def lognpdf(x, mu, sigma):
    shape  = sigma
    loc    = 0
    scale  = np.exp(mu)
    return lognorm.pdf(x, shape, loc, scale)

In [ ]:
count = 1
for data in data_all:
#     print(data)
    if count < 3:
        count += 1
        continue
    
    gene = data[data>=0]
    k = int(0.95*len(gene))
    gene = gene[np.argpartition(gene, k)[:k]]
    gene = np.sort(gene)
    
    data_mean = np.mean(gene);
    data_var = np.var(gene);
    data_noise = data_var/(data_mean**2);
    
    statis_data = utils.statisData(gene);
    
    rho = lambda s: np.sqrt(
                np.sum(
                    np.log(statis_data/(s))**2
                )
    )
    
    
    f = lambda k: utils.statisGTM(k,4)
    N = 1000 # the number of particles
    T = 5 # number of rounds
    epsilon = 1# a sequence of discrepancy acceptance thresholds
    
#     prior = lambda: np.array([
#                 random.uniform(0,5), # kon ~ U[0,5]
#                 loguniform(10**-1, 10**1).rvs(), # ron ~ logU[-1,1] base 10
#                 random.uniform(0,5), # koff ~ U[0,5]
#                 loguniform(10**-1, 10**1).rvs(), # roff ~ logU[-1,1] base 10 
#                 random.uniform(0,50), # rsyn ~ U[0,50] ?? in paper assumes upper bound 50, but in code is 100
#                 1]) # rdeg=1
    prior = lambda: np.array([
                5 * np.random.uniform(), # kon ~ U[0,5]
                10**random.uniform(-1,1), # ron ~ logU[-1,1] base 10
                5 * np.random.uniform(), # koff ~ U[0,5]
                10**random.uniform(-1,1), # roff ~ logU[-1,1] base 10 
                100 * np.random.uniform(), # rsyn ~ U[0,50] ?? in paper assumes upper bound 50, but in code is 100
                1]) # rdeg=1
    
    proposal_sigma = 0.2
    
    proposal = lambda x: np.random.lognormal(x,proposal_sigma)
    
    proposal_pdf = lambda kon_post,kon_prior,ron_post,ron_prior,koff_post,koff_prior,roff_post,roff_prior,mu_post,mu_prior: \
    lognpdf(mu_post,np.log(mu_prior),proposal_sigma) * lognpdf(kon_post,np.log(kon_prior),proposal_sigma) \
    * lognpdf(ron_post,np.log(ron_prior),proposal_sigma) * lognpdf(koff_post,np.log(koff_prior),proposal_sigma) \
    * lognpdf(roff_post,np.log(roff_prior),proposal_sigma) 
    
    
#     result, flag = abcSampler.ABCRejectionSampler(N,prior,f,rho,epsilon,T,count)
#     result = np.load('data/gene_1_prior.npy', allow_pickle='True')
    
#     result_true = np.load('data/gene_1_prior.npy', allow_pickle='True')
    
#     print(result_true)

    result, flag = abcSampler.ABCSMCSampler(N,prior,f,rho,epsilon,T,proposal,proposal_pdf,count)
    

    # Save posterior
    np.save(f'data/posterior/gene_{count}.npy', np.array([count, gene, result], dtype=object))
    
#     if count > 1: break
    count+=1

Processing Prior: Gene = 3
Processing Posterior: Gene = 3, T = 2
Processing Posterior: Gene = 3, T = 3
Processing Posterior: Gene = 3, T = 4
Processing Posterior: Gene = 3, T = 5
Processing Prior: Gene = 4
Processing Posterior: Gene = 4, T = 2
Processing Posterior: Gene = 4, T = 3
Processing Posterior: Gene = 4, T = 4
Processing Posterior: Gene = 4, T = 5
Processing Prior: Gene = 5
Processing Posterior: Gene = 5, T = 2
Processing Posterior: Gene = 5, T = 3
Processing Posterior: Gene = 5, T = 4
Processing Posterior: Gene = 5, T = 5
Processing Prior: Gene = 6
Processing Posterior: Gene = 6, T = 2
Processing Posterior: Gene = 6, T = 3
Processing Posterior: Gene = 6, T = 4
Processing Posterior: Gene = 6, T = 5
Processing Prior: Gene = 7
Processing Posterior: Gene = 7, T = 2
Processing Posterior: Gene = 7, T = 3
Processing Posterior: Gene = 7, T = 4
Processing Posterior: Gene = 7, T = 5
Processing Prior: Gene = 8
Processing Posterior: Gene = 8, T = 2
Processing Posterior: Gene = 8, T = 3
Pr

In [ ]:
data_mean

In [ ]:
data_var

In [ ]:
data_noise

In [ ]:
result[0,0]

In [ ]:
kon_post = 5.30753304760157;
kon_prior = 3.152078539638027;
ron_post = 8.70731191726281;
ron_prior = 9.601789150560395;
koff_post = 3.5623878531212596;
koff_prior = 1.1835791487917824;
roff_post = 4.520511228784241;
roff_prior = 2.138386913508971;
mu_post = 101.10914066538857;
mu_prior = 95.27428156492888;

proposal_pdf(kon_post,kon_prior,ron_post,ron_prior,koff_post,koff_prior,roff_post,roff_prior,mu_post,mu_prior) 

In [ ]:
5 * np.random.uniform()